In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

## FUNCOES

In [2]:
def carregar_imagens(pasta):
    imagens = {}
    for nome_ficheiro in os.listdir(pasta):
        if nome_ficheiro.lower().endswith(('.jpg', '.png', '.jpeg')):
            caminho = os.path.join(pasta, nome_ficheiro)
            imagem = cv2.imread(caminho)
            imagens[nome_ficheiro] = imagem
    return imagens

In [3]:
def converter_para_rgb(imagem):
    return cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

In [4]:
def refinar_mascara(mascara):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    kernel2= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    kernel3= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30, 30))
    kernel4= cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))

    #mascara_erosao = cv2.erode(mascara, kernel1)
    #mascara_refinada = cv2.dilate(mascara_erosao, kernel2)

    mascara_close = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel1)
    mascara_open = cv2.morphologyEx(mascara_close, cv2.MORPH_OPEN, kernel2)
    mascara_erosao = cv2.erode(mascara_open, kernel3)
    mascara_final = cv2.dilate(mascara_erosao, kernel4)

    return mascara_final

In [5]:
def aplicar_mascara(imagem, mascara):
    return cv2.bitwise_and(imagem, imagem, mask=mascara)

In [6]:
def gravar_imagem(imagem, nome_arquivo):
    cv2.imwrite(nome_arquivo, imagem)

In [7]:
def criar_mascara_por_cor(imagem_rgb, cor_inferior, cor_superior):
    mascara = cv2.inRange(imagem_rgb, cor_inferior, cor_superior)
    return mascara

In [8]:
def exibir_imagem_cv2(titulo, imagem):
    cv2.imshow(titulo, imagem)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
def extracao_propriedades(contorno):
    
    #areas
    area = cv2.contourArea(contorno)

    rect = cv2.minAreaRect(contorno)
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    
    #racio de area
    area_rect = cv2.contourArea(box)
    if area_rect == 0:
        racio = 0
    else:
        racio = (area / area_rect) * 100
        
    (x, y), (w, h), a = rect
    
    if min(w, h) == 0:
        aspect_ratio = 0
    else:
        aspect_ratio = max(w, h) / min(w, h)

    print("aspect_ratio:", aspect_ratio)
    
    return area, aspect_ratio, box, racio

In [10]:
def classificacao_peca(area, aspect_ratio, racio, min_area):
    
    if area > min_area:

        #area diferenca
        if racio < 89:
            return "indefinido"
    
        #aspect ratio
        if 0.8 < aspect_ratio < 1.4:
            return "2x2"
        elif 1.8 < aspect_ratio < 2.6:
            return"2x4"
        elif 2.8 < aspect_ratio < 3.6:
            return "2x6"
        elif 3.8 < aspect_ratio < 5.1:
            return "2x8"
        else:
            return "indefinido"

# EXECUÇÃO

In [11]:
pasta_treino = "./imagens"
imagens = carregar_imagens(pasta_treino)
print(len(imagens), "imagens carregadas.")

14 imagens carregadas.


In [12]:
nomes = list(imagens.keys())

for i in range(len(imagens)):
    #selecao da imagem
    nome = nomes[i]
    img = imagens[nome] 

    #conversao para rgb
    rgb_img = converter_para_rgb(img)

    #definicao dos intervalos de cor do fundo
    fundo_inferior = np.array([90, 0, 0])      
    fundo_superior = np.array([255, 255, 255])

    #criacao e aplocacao da mascara
    mascara_fundo = criar_mascara_por_cor(rgb_img, fundo_inferior, fundo_superior)
    mascara_refinada = refinar_mascara(mascara_fundo)
    imagem_isolada = aplicar_mascara(rgb_img, mascara_refinada)


    #desenho dos contornos dos objectos
    contornos, _ = cv2.findContours(mascara_refinada,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    #desenho dos contornos na imagem original
    imagem_com_contornos = img.copy()
    cv2.drawContours(imagem_com_contornos, contornos, -1, (0, 255, 0), 2)

    #exibicao das imagens
    #exibir_imagem_cv2("Imagem original", img)
    #exibir_imagem_cv2("Máscara refinada", mascara_refinada)
    #exibir_imagem_cv2("Contornos na imagem original", imagem_com_contornos)


    #gravacao das imagens
    #gravar_imagem(cv2.cvtColor(mascara_refinada, cv2.COLOR_GRAY2BGR), "mascara_refinada.jpg")
    #gravar_imagem(cv2.cvtColor(imagem_isolada, cv2.COLOR_RGB2BGR), "imagem_isolada.jpg")
    #gravar_imagem(imagem_com_contornos, "imagem_com_contornos.jpg")

    pecas_classificadas = []

    image_h = img.shape[0]
    image_w = img.shape[1]

    min_area = 1500

    for contorno in contornos:
        
        x, y, w, h = cv2.boundingRect(contorno)
        margem = 10
        if x <= margem or y <= margem or x + w >= image_w - margem or y + h >= image_h - margem:
            continue
        else:
            area, aspect_ratio, box, racio = extracao_propriedades(contorno)
            cv2.drawContours(imagem_com_contornos, [box], 0, (255, 0, 0), 2)

            tipo = classificacao_peca(area, aspect_ratio, racio, min_area)

            pecas_classificadas.append((tipo, area, aspect_ratio))

            x, y, w, h = cv2.boundingRect(contorno)
            posicao_texto = (x, y)
            posicao_texto_2 = (x, y - 50)  

            cv2.putText(imagem_com_contornos, tipo, posicao_texto, cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            #if area > min_area:
                #cv2.putText(imagem_com_contornos, str(aspect_ratio), posicao_texto_2, cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)

    gravar_imagem(cv2.cvtColor(mascara_refinada, cv2.COLOR_GRAY2BGR), f"resultados/mascara_refinada_{i}.jpg")
    gravar_imagem(cv2.cvtColor(imagem_isolada, cv2.COLOR_RGB2BGR), f"resultados/imagem_isolada_{i}.jpg")
    gravar_imagem(imagem_com_contornos, f"resultados/imagem_com_contornos_{i}.jpg")

    print(pecas_classificadas)

aspect_ratio: 1.0185459462757018
aspect_ratio: 1.6981133559283204
aspect_ratio: 2.2287120806750362
aspect_ratio: 1.6901095462486133
aspect_ratio: 2.262927433253323
aspect_ratio: 2.1747574239140945
aspect_ratio: 4.72916654163348
aspect_ratio: 1.0341685015748046
aspect_ratio: 1.0486331140075251
[('2x2', 6258.5, 1.0185459462757018), ('indefinido', 14304.5, 1.6981133559283204), ('2x4', 11897.0, 2.2287120806750362), ('indefinido', 9194.5, 1.6901095462486133), ('2x4', 11580.5, 2.262927433253323), ('2x4', 12463.5, 2.1747574239140945), ('2x8', 23246.0, 4.72916654163348), ('2x2', 6614.0, 1.0341685015748046), ('2x2', 6092.0, 1.0486331140075251)]
aspect_ratio: 2.2030567736474285
aspect_ratio: 1.4093438095871682
aspect_ratio: 2.3090184987144515
aspect_ratio: 4.855026217104629
aspect_ratio: 1.0203001179796636
aspect_ratio: 1.0575540322930164
[('2x4', 11884.5, 2.2030567736474285), ('indefinido', 7019.0, 1.4093438095871682), ('2x4', 12058.5, 2.3090184987144515), ('2x8', 22721.5, 4.855026217104629), (